# Data Cleansing and Imputation Decisions

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import t, norm
from math import atanh, pow
from numpy import tanh
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import KNNImputer

In [ ]:
df = pd.read_csv("5.4 full_data.csv")

In [ ]:
def rz_ci(self, r, n):

        zr_se = pow(1/(n - 3), .5)

        moe = norm.ppf(1 - (1 - self._confidence)/float(2)) * zr_se

        zu = atanh(r) + moe

        zl = atanh(r) - moe

        return tanh((zl, zu))

In [ ]:
df.reset_index(inplace= True,drop = True)

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
from pyMechkar.analysis import Table1

In [ ]:
res = Table1(data=df, categorize=True,catmiss=True)

In [ ]:
res[(res.Categories !='Mean (SD)') & (res.Categories != 'Median (IQR)')].Variables.unique()

In [ ]:
cv = ['GENDER', 'ETHNICITY', 'EDUCATION','FAM_INCOME','pulse_type','sw_smoker_cat', 'sw_cannabis', 'sw_drugs',
     'sw_diab', 'sw_prediab', 'sw_diabrisk', 'sw_insulin','sw_oral_antidiab', 'diab_retinopathy', 'CAD', 'CHF', 'MI',
       'STROKE', 'FAMILY_DIAB', 'sw_asthma_ever', 'sw_overweight','sw_malignancy', 'RENAL', 'DIALYSIS',]

In [ ]:
for v in cv:
    df[v] = df[v].astype('category')


In [ ]:
df.dtypes

In [ ]:
df = df.drop('SEQN',1)

## Outliers

In [ ]:
df.WAIST_CIRC.describe()


In [ ]:

########################
### Outliers - Python
########################

######### 
'''
Useful links:

Two-sample Kolmogorov-Smirnov Test in Python Scipy: https://stackoverflow.com/questions/10884668/two-sample-kolmogorov-smirnov-test-in-python-scipy
Fisher r-to-z transformation calculator: http://vassarstats.net/rdiff.html
build and fill pandas dataframe from for loop (second answer): https://stackoverflow.com/questions/28056171/how-to-build-and-fill-pandas-dataframe-from-for-loop
'''


### outlier matrix
def getOutliersMatrix(df, threshold=1.5):
    numeric_columns_names = df.select_dtypes("number").columns
    numeric_df = df[[name for name in numeric_columns_names]]
    
    Q1 = numeric_df.quantile(0.25)
    Q3 = numeric_df.quantile(0.75)
    IQR = Q3 - Q1
    
    outdata = (numeric_df < (Q1 - 1.5 * IQR)) | (numeric_df > (Q3 + 1.5 * IQR))
    
    for name in numeric_df.columns:
        outdata.loc[(outdata[name] == True), name] = 1
        outdata.loc[(outdata[name] == False), name] = 0
    
    return outdata


### differences between two correlation in Python: https://github.com/psinger/CorrelationStats


def independent_corr(xy, ab, n, n2 = None, twotailed=True, conf_level=0.95, method='fisher'):
    """
    Calculates the statistic significance between two independent correlation coefficients
    @param xy: correlation coefficient between x and y
    @param xz: correlation coefficient between a and b
    @param n: number of elements in xy
    @param n2: number of elements in ab (if distinct from n)
    @param twotailed: whether to calculate a one or two tailed test, only works for 'fisher' method
    @param conf_level: confidence level, only works for 'zou' method
    @param method: defines the method uses, 'fisher' or 'zou'
    @return: z and p-val
    """

    if method == 'fisher':
        xy_z = 0.5 * np.log((1 + xy)/(1 - xy))
        ab_z = 0.5 * np.log((1 + ab)/(1 - ab))
        if n2 is None:
            n2 = n

        se_diff_r = np.sqrt(1/(n - 3) + 1/(n2 - 3))
        diff = xy_z - ab_z
        z = abs(diff / se_diff_r)
        p = (1 - norm.cdf(z))
        if twotailed:
            p *= 2

        return z, p
    elif method == 'zou':
        L1 = rz_ci(xy, n, conf_level=conf_level)[0]
        U1 = rz_ci(xy, n, conf_level=conf_level)[1]
        L2 = rz_ci(ab, n2, conf_level=conf_level)[0]
        U2 = rz_ci(ab, n2, conf_level=conf_level)[1]
        lower = xy - ab - pow((pow((xy - L1), 2) + pow((U2 - ab), 2)), 0.5)
        upper = xy - ab + pow((pow((U1 - xy), 2) + pow((ab - L2), 2)), 0.5)
        return lower, upper
    else:
        raise Exception('Wrong method!')

def get_df_without_outliers(df, threshold=1.5):
    numeric_columns_names = df.select_dtypes("number").columns
    numeric_df = df[[name for name in numeric_columns_names]]
    
    Q1 = numeric_df.quantile(0.25)
    Q3 = numeric_df.quantile(0.75)
    IQR = Q3 - Q1

    no_outliers_df = numeric_df[(numeric_df >= (Q1 - 1.5 * IQR)) & (numeric_df <= (Q3 + 1.5 * IQR))]
    
    return no_outliers_df


def OutliersRemoveDecisions(df, y_label, threshold=1.5, significant_level=0.05):
    numeric_columns_names = df.select_dtypes("number").columns
    numeric_df = df[[name for name in numeric_columns_names]]
    
    with_outliers_df = numeric_df
    without_outliers_df = get_df_without_outliers(numeric_df, threshold)
    
    
    distribution_change_df = []

    for name in numeric_columns_names:
        print(name)
        with_outliers_series = with_outliers_df[name]
        without_outliers_series = without_outliers_df[name]
    
        ks_result = stats.ks_2samp(with_outliers_series, without_outliers_series)
        
        x_outliers = with_outliers_series
        x_no_outliers = without_outliers_series
        
        xy = x_outliers.corr(y_label, method='spearman')
        ab = x_no_outliers.corr(y_label, method='spearman')
                
        n = x_outliers.shape[0]
        n2 = x_no_outliers.shape[0]
        
        ftz_result = independent_corr(xy, ab, n, n2, method='fisher')
        totout = np.sum(with_outliers_series.count()) - np.sum(without_outliers_series.count())
        
        
        distribution_change_df.append(
            {
                'variable' : name,
                'outliers_number': '{} ({})'.format(totout, np.round(totout*100/n,2)),
                'distribution_change': '+' if ks_result[1] < significant_level else '-', #  ks_result[1] -> gives us the p-value
                'correlation_changed': '+' if ftz_result[1] < significant_level else '-', #  ftz_result[1] -> gives us the p-value
                'drop': 'no' if ((ks_result[1] < significant_level) & (ftz_result[1] < significant_level)) else 'yes' 
            }
        )
    
    return pd.DataFrame(distribution_change_df)


In [ ]:
out = OutliersRemoveDecisions(df,df.WAIST_CIRC)

In [ ]:
out

## Missing Values


In [ ]:
################################
### Missing values - Python
################################

def get_missings_matrix(df):
    missings_matrix_df = df.isna().astype(int)
    return missings_matrix_df

def MissingMechanism(df, significant_level=0.05):
    missing_matt = get_missings_matrix(df)
    numeric_columns_names = df.select_dtypes("number").columns
    distribution_change_df = pd.DataFrame(columns=['var_1_missing','var_2','missings_number','distribution_changed','imputation'])
    #i = 0
    for numeric_missing_column_name in numeric_columns_names: # -> only numeric variables that have missing values 
        #print(numeric_missing_column_name)
        df_without_na = df[missing_matt[numeric_missing_column_name] == 0]
        for numeric_column_name in numeric_columns_names: # -> all numeric variables in data: with and without missing values
            if numeric_missing_column_name != numeric_column_name:
                series_with_na = df[numeric_column_name]
                series_without_na = df_without_na[numeric_column_name]
                ks_result = stats.ks_2samp(series_with_na, series_without_na)
                #print(numeric_missing_column_name, numeric_column_name, round(ks_result[1],3))
                distribution_change_df = distribution_change_df.append(
                    {
                        'var_1_missing' : numeric_missing_column_name,
                        'var_2': numeric_column_name,
                        'missings_number': sum(missing_matt[numeric_missing_column_name]), 
                        'distribution_changed': '+' if ks_result[1] < significant_level else '-', # ks_result[1] -> p-value
                        'imputation': 'no' if (ks_result[1] < significant_level) else 'yes' 
                    }, ignore_index=True
                )
                #print(i)
           # i = i + 1

    return pd.DataFrame(distribution_change_df)



In [ ]:
df.isnull().sum()

In [ ]:
rowmis = df.isnull().sum(axis=1)/81
rowmis[rowmis>=0.5].sort_values(ascending=False)

In [ ]:
### % of rows with 50% or more missing
rowmis[rowmis>=0.5].count()/df.shape[0]

In [ ]:
df.drop(rowmis[rowmis>=0.5].index).shape

In [ ]:
df = df.drop(rowmis[rowmis>=0.5].index)

In [ ]:
rowmis = df.isnull().sum(axis=1)/81
rowmis.sort_values(ascending=False)

In [ ]:
mis = MissingMechanism(df)

In [ ]:
df.reset_index(inplace= True , drop = True)

In [ ]:
mis

In [ ]:
min(mis.imputation)

In [ ]:
## If no appears at least once for a variable with missings values, then we can't do imputation

impute = mis.loc[mis['missings_number']>0,['var_1_missing','imputation']].groupby(['var_1_missing']).agg('min').reset_index()
impute

In [ ]:
## imputate
yes=impute[impute.imputation=='yes'].reset_index(drop=True)

In [ ]:
df[yes["var_1_missing"]]

In [ ]:
## can't do imputation -> convert to categories
no=impute[impute.imputation=='no']

In [ ]:
df[no["var_1_missing"]]

In [ ]:

(df["ALBUMIN"].isnull().sum()*100)/df.shape[0]

In [ ]:
sum = list()
for val in impute["var_1_missing"]:
    sum.append(("name: " +val ,(df[val].isnull().sum()*100)/df.shape[0]))



In [ ]:
for i in range(len(sum)):
    if sum[i][1]>40:
        print(sum[i])


In [ ]:
names = ('Glucose_basal','HOMA_IR','HOMA_beta','Insulin_basal','LDL','age_first_smoke')

In [ ]:
print(df['age_first_smoke'])

In [ ]:
df['FERRITIN'].fillna(1,inplace = True)

In [ ]:
df['FERRITIN']

In [ ]:
for i in range(len(df['FERRITIN'])):
    if df['FERRITIN'][i]!=1:
        df['FERRITIN'][i]=0

In [ ]:
df['FERRITIN'].value_counts()

In [ ]:
df_temp = pd.DataFrame()
for val in yes["var_1_missing"]:
    df_temp[val] = df[val] 

In [ ]:
df_temp.isnull().sum()

In [ ]:
impute_knn= KNNImputer()
df_temp=pd.DataFrame(impute_knn.fit_transform(df_temp))

In [ ]:
df_temp.isnull().sum()

In [ ]:
j=0
for val in yes["var_1_missing"]:
    for i in range(len(df_temp[j])):
         df[val][i] = df_temp[j][i] 
    j+=1

In [ ]:
yes

In [ ]:
for val in yes["var_1_missing"]:
    print(val , df[val].isnull().sum())

In [ ]:
names

In [ ]:
df['age_first_smoke_na'] = df['age_first_smoke'].isnull().astype(int)

In [ ]:
df.shape

In [ ]:
df.to_csv("data 31.05.csv")

# progress 6/7/22

# Part 1 - Feature Engineering

In [20]:
df =pd.read_csv("5.4 full_data.csv")

In [21]:
df

,SEQN,GENDER,AGE,ETHNICITY,EDUCATION,HOUSE_INDIV,FAMILY_SIZE,FAM_INCOME,INTERVIE_wgt,EXAM_wgt,...,FAMILY_DIAB,sw_asthma_ever,sw_overweight,sw_malignancy,RENAL,DIALYSIS,HDL,LDL,FERRITIN,FOLATE
0,41475.0,2,62,5,3,2,2,6.0,59356.356426,60045.772497,...,1,1,1,2,2,0,47.0,NaN,NaN,24.0
1,41477.0,1,71,3,3,2,2,5.0,9935.266183,10074.150074,...,2,2,2,2,1,0,34.0,NaN,NaN,29.0
2,41479.0,1,52,1,1,5,5,8.0,8727.797555,9234.055759,...,2,2,2,2,2,0,47.0,121.0,NaN,36.9
3,41481.0,1,21,4,3,4,4,6.0,24342.505253,24655.376656,...,2,2,2,2,2,0,NaN,NaN,NaN,NaN
4,41482.0,1,64,1,2,6,6,15.0,9811.075078,11602.178638,...,1,2,1,2,2,0,43.0,NaN,NaN,42.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29251,102949.0,1,33,3,3,5,5,6.0,29334.650205,30287.188731,...,2,2,2,2,2,0,73.0,NaN,219.00,477.0
29252,102952.0,2,70,5,3,2,2,4.0,16896.276203,18338.711104,...,2,2,2,2,2,0,60.0,43.0,135.00,NaN
29253,102953.0,1,42,1,3,1,1,12.0,61630.380013,63661.951573,...,2,1,1,2,2,0,49.0,NaN,120.00,NaN
29254,102954.0,2,41,4,5,7,7,10.0,17160.895269,17694.783346,...,1,2,1,2,2,0,54.0,108.0,1.99,728.0


In [22]:
fe_names= ['BUN_CREATININE',
'ALBUMIN_CREATININE',
'URIC_CREATININE',
'URIC_ALBUMIN', 
'PROTEIN_CREATININE', 
'PROTEIN_ALBUMIN',
'PROTEIN_GLOBULIN',
'GLOBULIN_ALBUMIN',
'IRON_CREATININE',
'PROTEIN_CALCIUM',
'CALCIUM_PHOSPHORUS',
'PROTEIN_PHOSPHORUS',
'IRON_PHOSPHORUS',
'IRON_FOLATE',
'PULSE_CALCIUM',
'PULSE_PHOSPHORUS',
'URIC_PROTEIN',
'PHOSPHORUS_TRIGLYCERIDS',
'TRIGLYCERIDS_CHOLESTEROL',
'CHOLESTEROL_HDL',
'HDL_TRIGLYCERIDS',
'DELTA_BP_second'
]

In [23]:
df['BUN_CREATININE']=np.nan
for i in range(len(df)):
    df['BUN_CREATININE'][i]= df['BUN'][i]/df['CREATININE'][i]*100
cat = pd.cut(df.BUN_CREATININE,bins=[-9999,4.8,7.6,9999],labels=['<4.8','4.8-7.6','>7.6'])
df['BUN_CREATININE'] =cat

<ipython-input-23-e8b39054cd7d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['BUN_CREATININE'][i]= df['BUN'][i]/df['CREATININE'][i]*100


In [24]:
df['ALBUMIN_CREATININE']=np.nan
for i in range(len(df)):
    df['ALBUMIN_CREATININE'][i]= df['ALBUMIN'][i]/df['CREATININE'][i]*100
cat = pd.cut(df.ALBUMIN_CREATININE,bins=[-9999,48,69,9999],labels=['<48','48-69','>69'])
df['ALBUMIN_CREATININE'] =cat

<ipython-input-24-44c21a9ce13e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ALBUMIN_CREATININE'][i]= df['ALBUMIN'][i]/df['CREATININE'][i]*100


In [25]:
df['URIC_CREATININE']=np.nan
for i in range(len(df)):
    df['URIC_CREATININE'][i]= df['URIC_ACID'][i]/df['CREATININE'][i]
cat = pd.cut(df.URIC_CREATININE,bins=[-9999,3.56,5.0,9999],labels=['<3.56','3.56-5.0','>5.0'])
df['URIC_CREATININE'] =cat

<ipython-input-25-8b292b2efe45>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['URIC_CREATININE'][i]= df['URIC_ACID'][i]/df['CREATININE'][i]


In [26]:
df['URIC_ALBUMIN']=np.nan 
for i in range(len(df)):
    df['URIC_ALBUMIN'][i]= df['URIC_ACID'][i]/df['ALBUMIN'][i]
cat = pd.cut(df.URIC_ALBUMIN,bins=[-9999,6.2,8.9,9999],labels=['<6.2','6.2-8.9','>8.9'])
df['URIC_ALBUMIN'] =cat

<ipython-input-26-e0fbd2fa7229>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['URIC_ALBUMIN'][i]= df['URIC_ACID'][i]/df['ALBUMIN'][i]


In [27]:
df['PROTEIN_CREATININE']=np.nan
for i in range(len(df)):
    df['PROTEIN_CREATININE'][i]= df['PROTEIN'][i]/df['CREATININE'][i]*10
cat = pd.cut(df.PROTEIN_CREATININE,bins=[-9999,8.1,11.5,9999],labels=['<8.1','8.1-11.5','>11.5'])
df['PROTEIN_CREATININE'] =cat

<ipython-input-27-ff35168b749e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PROTEIN_CREATININE'][i]= df['PROTEIN'][i]/df['CREATININE'][i]*10


In [28]:
df['PROTEIN_ALBUMIN']=np.nan
for i in range(len(df)):
    df['PROTEIN_ALBUMIN'][i]= df['PROTEIN'][i]/df['ALBUMIN'][i]*10
cat = pd.cut(df.PROTEIN_ALBUMIN,bins=[-9999,16,17.75,9999],labels=['<16','16-17.75','>17.75'])
df['PROTEIN_ALBUMIN'] =cat

<ipython-input-28-8b917219c81c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PROTEIN_ALBUMIN'][i]= df['PROTEIN'][i]/df['ALBUMIN'][i]*10


In [29]:
df['PROTEIN_GLOBULIN']=np.nan
for i in range(len(df)):
    df['PROTEIN_GLOBULIN'][i]= df['PROTEIN'][i]/df['GLOBULIN'][i]*10
cat = pd.cut(df.PROTEIN_GLOBULIN,bins=[-9999,23.0,26.6,9999],labels=['<23.0','23.0-26.6','>26.6'])
df['PROTEIN_GLOBULIN'] =cat

<ipython-input-29-f087371c7933>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PROTEIN_GLOBULIN'][i]= df['PROTEIN'][i]/df['GLOBULIN'][i]*10


In [30]:
df['GLOBULIN_ALBUMIN']=np.nan
for i in range(len(df)):
    df['GLOBULIN_ALBUMIN'][i]= df['GLOBULIN'][i]/df['ALBUMIN'][i]*100
cat = pd.cut(df.GLOBULIN_ALBUMIN,bins=[-9999,60,77.5,9999],labels=['<60','60-77.5','>77.5'])
df['GLOBULIN_ALBUMIN'] =cat

<ipython-input-30-483c0553ee1a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['GLOBULIN_ALBUMIN'][i]= df['GLOBULIN'][i]/df['ALBUMIN'][i]*100


In [31]:
df['IRON_CREATININE']=np.nan
for i in range(len(df)):
    df['IRON_CREATININE'][i]= df['IRON'][i]/df['CREATININE'][i]*100
cat = pd.cut(df.IRON_CREATININE,bins=[-9999,14,26,9999],labels=['<14','14-26','>26'])
df['IRON_CREATININE'] =cat

<ipython-input-31-55d9ffc3daad>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['IRON_CREATININE'][i]= df['IRON'][i]/df['CREATININE'][i]*100


In [32]:
df['PROTEIN_CALCIUM']=np.nan
for i in range(len(df)):
    df['PROTEIN_CALCIUM'][i]= df['PROTEIN'][i]/df['CALCIO'][i]
cat = pd.cut(df.PROTEIN_CALCIUM,bins=[-9999,29.3,31.75,9999],labels=['<29.3','29.3-31.75','>31.75'])
df['PROTEIN_CALCIUM'] =cat

<ipython-input-32-9add1c8e204d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PROTEIN_CALCIUM'][i]= df['PROTEIN'][i]/df['CALCIO'][i]


In [33]:
df['CALCIUM_PHOSPHORUS']=np.nan
for i in range(len(df)):
    df['CALCIUM_PHOSPHORUS'][i]= df['CALCIO'][i]/df['PHOSPHORUS'][i]*10
cat = pd.cut(df.CALCIUM_PHOSPHORUS,bins=[-9999,18,21.7,9999],labels=['<18','18-21.7','>21.7'])
df['CALCIUM_PHOSPHORUS'] =cat

<ipython-input-33-1beb707f3c2d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CALCIUM_PHOSPHORUS'][i]= df['CALCIO'][i]/df['PHOSPHORUS'][i]*10


In [34]:
df['PROTEIN_PHOSPHORUS']=np.nan
for i in range(len(df)):
    df['PROTEIN_PHOSPHORUS'][i]= df['PROTEIN'][i]/df['PHOSPHORUS'][i]
cat = pd.cut(df.PROTEIN_PHOSPHORUS,bins=[-9999,54,67,9999],labels=['<54','54-67','>67'])
df['PROTEIN_PHOSPHORUS'] =cat

<ipython-input-34-d3628d343386>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PROTEIN_PHOSPHORUS'][i]= df['PROTEIN'][i]/df['PHOSPHORUS'][i]


In [35]:
df['IRON_PHOSPHORUS']=np.nan
for i in range(len(df)):
    df['IRON_PHOSPHORUS'][i]= df['IRON'][i]/df['PHOSPHORUS'][i]
cat = pd.cut(df.IRON_PHOSPHORUS,bins=[-9999,12,16.2,9999],labels=['<12','12-16.2','>16.2'])
df['IRON_PHOSPHORUS'] =cat

<ipython-input-35-d17f500fcec5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['IRON_PHOSPHORUS'][i]= df['IRON'][i]/df['PHOSPHORUS'][i]


In [36]:
df['IRON_FOLATE']=np.nan
for i in range(len(df)):
    df['IRON_FOLATE'][i]= df['IRON'][i]/df['FOLATE'][i]*100
cat = pd.cut(df.IRON_FOLATE,bins=[-9999,2.6,6.9,9999],labels=['<2.6','2.6-6.9','>6.9'])
df['IRON_FOLATE'] =cat

<ipython-input-36-e40a42213f0a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['IRON_FOLATE'][i]= df['IRON'][i]/df['FOLATE'][i]*100


In [37]:
df['PULSE_CALCIUM']=np.nan
for i in range(len(df)):
    df['PULSE_CALCIUM'][i]= df['pusle'][i]/df['CALCIO'][i]
cat = pd.cut(df.PULSE_CALCIUM,bins=[-9999,27.2,34.0,9999],labels=['<27.2','27.2-34.0','>34.0'])
df['PULSE_CALCIUM'] =cat

<ipython-input-37-e305448aaa4a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PULSE_CALCIUM'][i]= df['pusle'][i]/df['CALCIO'][i]


In [38]:
df['PULSE_PHOSPHORUS']=np.nan
for i in range(len(df)):
    df['PULSE_PHOSPHORUS'][i]= df['pusle'][i]/df['PHOSPHORUS'][i]
cat = pd.cut(df.PULSE_PHOSPHORUS,bins=[-9999,52,70,9999],labels=['<52','52-70','>70'])
df['PULSE_PHOSPHORUS'] =cat

<ipython-input-38-124ac0bc3be4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PULSE_PHOSPHORUS'][i]= df['pusle'][i]/df['PHOSPHORUS'][i]


In [39]:
df['URIC_PROTEIN']=np.nan
for i in range(len(df)):
    df['URIC_PROTEIN'][i]= df['URIC_ACID'][i]/df['PROTEIN'][i]
cat = pd.cut(df.URIC_PROTEIN,bins=[-9999,3.7,5.2,9999],labels=['<3.7','3.7-5.2','>5.2'])
df['URIC_PROTEIN'] =cat

<ipython-input-39-2f6b77b94b19>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['URIC_PROTEIN'][i]= df['URIC_ACID'][i]/df['PROTEIN'][i]


In [40]:
df['PHOSPHORUS_TRIGLYCERIDS']=np.nan
for i in range(len(df)):
    df['PHOSPHORUS_TRIGLYCERIDS'][i]= df['PHOSPHORUS'][i]/df['TRIGLYCERIDS'][i]*10
cat = pd.cut(df.PHOSPHORUS_TRIGLYCERIDS,bins=[-9999,5.6,13.1,9999],labels=['<5.6','5.6-13.1','>13.1'])
df['PHOSPHORUS_TRIGLYCERIDS'] =cat

<ipython-input-40-308d496e3602>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PHOSPHORUS_TRIGLYCERIDS'][i]= df['PHOSPHORUS'][i]/df['TRIGLYCERIDS'][i]*10


In [41]:
df['TRIGLYCERIDS_CHOLESTEROL']=np.nan
for i in range(len(df)):
    df['TRIGLYCERIDS_CHOLESTEROL'][i]= df['TRIGLYCERIDS'][i]/df['CHOLESTEROL'][i]*100
cat = pd.cut(df.TRIGLYCERIDS_CHOLESTEROL,bins=[-9999,19,42.2,9999],labels=['<19','19-42.2','>42.2'])
df['TRIGLYCERIDS_CHOLESTEROL'] =cat

<ipython-input-41-cfd3229eaaa2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TRIGLYCERIDS_CHOLESTEROL'][i]= df['TRIGLYCERIDS'][i]/df['CHOLESTEROL'][i]*100


In [42]:
df['CHOLESTEROL_HDL']=np.nan
for i in range(len(df)):
    df['CHOLESTEROL_HDL'][i]= df['CHOLESTEROL'][i]/df['HDL'][i]*100
cat = pd.cut(df.CHOLESTEROL_HDL,bins=[-9999,7.6,12.1,9999],labels=['<7.6','7.6-12.1','>12.1'])
df['CHOLESTEROL_HDL'] =cat

<ipython-input-42-66a7a41d8578>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CHOLESTEROL_HDL'][i]= df['CHOLESTEROL'][i]/df['HDL'][i]*100


In [43]:
df['HDL_TRIGLYCERIDS']=np.nan
for i in range(len(df)):
    df['HDL_TRIGLYCERIDS'][i]= df['HDL'][i]/df['TRIGLYCERIDS'][i]
cat = pd.cut(df.HDL_TRIGLYCERIDS,bins=[-9999,20.8,63.8,9999],labels=['<20.8','20.8-63.8','>63.8'])
df['HDL_TRIGLYCERIDS'] =cat

<ipython-input-43-98829da3b993>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['HDL_TRIGLYCERIDS'][i]= df['HDL'][i]/df['TRIGLYCERIDS'][i]


In [44]:
df['DELTA_BP_second']=np.nan
for i in range(len(df)):
    df['DELTA_BP_second'][i]= df['DBP_first'][i] - df['DBP_second'][i]
cat = pd.cut(df.DELTA_BP_second,bins=[-9999,40,60,9999],labels=['<40','40-60','>60'])
df['DELTA_BP_second'] =cat

<ipython-input-44-99585148b8dc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DELTA_BP_second'][i]= df['DBP_first'][i] - df['DBP_second'][i]


In [45]:
for val in fe_names:
   print(val)
   # print(df[val].dtype)
   df[val] = df[val].cat.add_categories('no')
   df[val].fillna('no', inplace =True) 


BUN_CREATININE
ALBUMIN_CREATININE
URIC_CREATININE
URIC_ALBUMIN
PROTEIN_CREATININE
PROTEIN_ALBUMIN
PROTEIN_GLOBULIN
GLOBULIN_ALBUMIN
IRON_CREATININE
PROTEIN_CALCIUM
CALCIUM_PHOSPHORUS
PROTEIN_PHOSPHORUS
IRON_PHOSPHORUS
IRON_FOLATE
PULSE_CALCIUM
PULSE_PHOSPHORUS
URIC_PROTEIN
PHOSPHORUS_TRIGLYCERIDS
TRIGLYCERIDS_CHOLESTEROL
CHOLESTEROL_HDL
HDL_TRIGLYCERIDS
DELTA_BP_second


In [46]:
df[val].value_counts()

40-60    13588
<40       7349
>60       6868
no        1451
Name: DELTA_BP_second, dtype: int64

In [47]:
ohe = OneHotEncoder(sparse=False)
j=0
for val in fe_names:
    print(val)
    array =ohe.fit_transform(df[[val]])
    df_array = pd.DataFrame(array)
    for i in df_array.columns:
        name = val+'_'+str(j)
        print(name)
        print(df_array[i])
        df[name]= df_array[i]
        j+=1
    j=0

BUN_CREATININE
BUN_CREATININE_0
0        0.0
1        0.0
2        1.0
3        0.0
4        1.0
        ... 
29251    0.0
29252    1.0
29253    1.0
29254    0.0
29255    1.0
Name: 0, Length: 29256, dtype: float64
BUN_CREATININE_1
0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
29251    0.0
29252    0.0
29253    0.0
29254    1.0
29255    0.0
Name: 1, Length: 29256, dtype: float64
BUN_CREATININE_2
0        1.0
1        1.0
2        0.0
3        0.0
4        0.0
        ... 
29251    1.0
29252    0.0
29253    0.0
29254    0.0
29255    0.0
Name: 2, Length: 29256, dtype: float64
BUN_CREATININE_3
0        0.0
1        0.0
2        0.0
3        1.0
4        0.0
        ... 
29251    0.0
29252    0.0
29253    0.0
29254    0.0
29255    0.0
Name: 3, Length: 29256, dtype: float64
ALBUMIN_CREATININE
ALBUMIN_CREATININE_0
0        0.0
1        1.0
2        1.0
3        0.0
4        0.0
        ... 
29251    1.0
29252    0.0
29253    1.0
29254    1.0
29255    1.0
Name: 

In [48]:
df.to_csv("after_FE.csv")

# Part 2 - Categories

In [49]:
df = pd.read_csv("after_FE.csv")

In [50]:
# names = ['FAM_INCOME','pusle','age_first_smoke','SBP_first','SBP_second','DBP_first','DBP_second','ALBUMIN','BUN','CALCIO','CHOLESTEROL','CREATININE','GLUCOSE','IRON','PHOSPHORUS','BILIRUBIN','PROTEIN','TRIGLYCERIDS','URIC_ACID','GLOBULIN','Glucose_basal','Insulin_basal','HOMA_beta','HOMA_IR','HDL','LDL','FOLATE']
names = ['pusle','SBP_first','SBP_second','DBP_first','DBP_second','ALBUMIN','BUN','CALCIO','CHOLESTEROL','CREATININE','GLUCOSE','IRON','PHOSPHORUS','BILIRUBIN','PROTEIN','TRIGLYCERIDS','URIC_ACID','GLOBULIN','Glucose_basal','Insulin_basal','HOMA_beta','HOMA_IR','HDL','LDL','FOLATE']

In [51]:
for val in names:
    print(df[val].isna().sum())

808
2015
2015
1451
1451
1522
1526
1555
1529
1524
1524
1559
1529
1545
1562
1486
1531
1563
16480
19024
19041
19041
1437
16096
2977


In [52]:
df['FAM_INCOME'].fillna(99,inplace=True)

In [53]:
df['age_first_smoke'].fillna(0,inplace=True)

In [54]:
for i in range(len(df)):
    if df['age_first_smoke'][i] > 0:
        if (df['AGE'][i] - df['age_first_smoke'][i]) < 0:
            # df['age_first_smoke'][i] =df['AGE'][i]-df['age_first_smoke'][i]
            df['age_first_smoke'][i]=0
        else:
            df['age_first_smoke'][i] =df['AGE'][i]-df['age_first_smoke'][i]
    else:
        df['age_first_smoke'][i]=0

<ipython-input-54-4ce10b9903cf>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['age_first_smoke'][i]=0
<ipython-input-54-4ce10b9903cf>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['age_first_smoke'][i] =df['AGE'][i]-df['age_first_smoke'][i]
<ipython-input-54-4ce10b9903cf>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['age_first_smoke'][i]=0


In [55]:
df['age_first_smoke'].value_counts()

0.0     16821
33.0      276
42.0      274
41.0      267
44.0      266
        ...  
66.0       13
68.0        8
67.0        8
69.0        3
70.0        2
Name: age_first_smoke, Length: 71, dtype: int64

In [56]:
cat = pd.cut(df.pusle,bins=[-9999,50,59,69,79,89,99,9999],labels=['<50', '50-59','60-69','70-79','80-89','90-99','100+'])
df['pusle'] =cat

In [57]:
cat = pd.cut(df.SBP_first,bins=[-9999,120,129,130,139,9999],labels=['<120', '120-129','130','139','140+'])
df['SBP_first'] =cat

In [58]:
cat = pd.cut(df.SBP_second,bins=[-9999,120,129,130,139,9999],labels=['<120', '120-129','130','139','140+'])
df['SBP_second'] =cat

In [59]:
cat = pd.cut(df.DBP_first,bins=[-9999,80,89,9999],labels=['<80','80-89','90+'])
df['DBP_first'] =cat

In [60]:
cat = pd.cut(df.DBP_second,bins=[-9999,80,89,9999],labels=['<80','80-89','90+'])
df['DBP_second'] =cat

In [61]:
cat = pd.cut(df.ALBUMIN,bins=[-9999,34,9999],labels=['<34','>=34'])
df['ALBUMIN'] =cat

In [62]:
cat = pd.cut(df.BUN,bins=[-9999,7,21,30,9999],labels=['<7','7-21','22-30','>30'])
df['BUN'] =cat

In [63]:
cat = pd.cut(df.CALCIO,bins=[-9999,35,9999],labels=['<35','35+'])
df['CALCIO'] =cat

In [64]:
cat = pd.cut(df.CHOLESTEROL,bins=[-9999,5.2,6.2,9999],labels=['<5.2', '5.3-6.2','>6.2'])
df['CHOLESTEROL'] =cat

In [65]:
cat = pd.cut(df.CREATININE,bins=[-9999,62,115,9999],labels=['62','62-115','>115'])
df['CREATININE'] =cat

In [66]:
cat = pd.cut(df.GLUCOSE,bins=[-9999,3.9,5.5,6.98,7.76,11.0,9999],labels=['<3.9','3.9-5.5','5.6-6.98','6.99-7.76','7.77-11.0','>11.1'])
df['GLUCOSE'] =cat

In [67]:
cat = pd.cut(df.IRON,bins=[-9999,10.5,12.4,22.9,25.9,9999],labels=['<10.5','10.5-12.4','12.5-22.9','23-25.9','>=26'])
df['IRON'] =cat

In [68]:
cat = pd.cut(df.PHOSPHORUS,bins=[-9999,1.12,1.45,9999],labels=['<1.12','1.12-1.45','>1.45'])
df['PHOSPHORUS'] =cat

In [69]:
cat = pd.cut(df.BILIRUBIN,bins=[-9999,20.5,9999],labels=['<20.5','>=20.5'])
df['BILIRUBIN'] =cat

In [70]:
cat = pd.cut(df.PROTEIN,bins=[-9999,60,83,9999],labels=['<60','60-83','>83'])
df['PROTEIN'] =cat

In [71]:
cat = pd.cut(df.TRIGLYCERIDS,bins=[-9999,1.69,2.25,9999],labels=['<1.69','1.69-2.25','>2.25'])
df['TRIGLYCERIDS'] =cat

In [72]:
cat = pd.cut(df.URIC_ACID,bins=[-9999,240,269.9,509.9,729.9,9999],labels=['<240','240-269.9','270-509.9','510-729.9','730+'])
df['URIC_ACID'] =cat

In [73]:
cat = pd.cut(df.GLOBULIN,bins=[-9999,26,41,9999],labels=['<26','26-41','>41'])
df['GLOBULIN'] =cat

In [74]:
cat = pd.cut(df.Glucose_basal,bins=[-9999,70,99,125,140,200,9999],labels=['<70','70-99','100-125','126-140','140-200','>200'])
df['Glucose_basal'] =cat

In [75]:
cat = pd.cut(df.Insulin_basal,bins=[-9999,30,9999],labels=['<30','>=30'])
df['Insulin_basal'] =cat

In [76]:
cat = pd.cut(df.HOMA_beta,bins=[-9999,135,9999],labels=['<135','>=135'])
df['HOMA_beta'] =cat

In [77]:
cat = pd.cut(df.HOMA_IR,bins=[-9999,2.5,9999],labels=['<2.5','>=2.5'])
df['HOMA_IR'] =cat

In [78]:
cat = pd.cut(df.HDL,bins=[-9999,40,60,9999],labels=['<40','40-60','>60'])
df['HDL'] =cat

In [79]:
cat = pd.cut(df.LDL,bins=[-9999, 70,99,129,159,9999],labels=['<70','70-99','100-129','130-159','>160'])
df['LDL'] =cat

In [80]:
cat = pd.cut(df.FOLATE,bins=[-9999,61.2,385.2,9999],labels=['<61.2','61.2-385.2','>385.2'])
df['FOLATE'] =cat

In [81]:
for val in names:
   print(val)
   # print(df[val].dtype)
   df[val] = df[val].cat.add_categories('no')
   df[val].fillna('no', inplace =True) 



pusle
SBP_first
SBP_second
DBP_first
DBP_second
ALBUMIN
BUN
CALCIO
CHOLESTEROL
CREATININE
GLUCOSE
IRON
PHOSPHORUS
BILIRUBIN
PROTEIN
TRIGLYCERIDS
URIC_ACID
GLOBULIN
Glucose_basal
Insulin_basal
HOMA_beta
HOMA_IR
HDL
LDL
FOLATE


In [82]:
df.drop('Unnamed: 0',inplace = True,axis =1 )

In [83]:
df.to_csv("Data_After_Change.csv")

In [84]:
df = pd.read_csv("Data_After_Change.csv")

In [85]:
df.drop('Unnamed: 0',inplace = True,axis =1 )

In [86]:
df

,SEQN,GENDER,AGE,ETHNICITY,EDUCATION,HOUSE_INDIV,FAMILY_SIZE,FAM_INCOME,INTERVIE_wgt,EXAM_wgt,...,CHOLESTEROL_HDL_2,CHOLESTEROL_HDL_3,HDL_TRIGLYCERIDS_0,HDL_TRIGLYCERIDS_1,HDL_TRIGLYCERIDS_2,HDL_TRIGLYCERIDS_3,DELTA_BP_second_0,DELTA_BP_second_1,DELTA_BP_second_2,DELTA_BP_second_3
0,41475.0,2,62,5,3,2,2,6.0,59356.356426,60045.772497,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,41477.0,1,71,3,3,2,2,5.0,9935.266183,10074.150074,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,41479.0,1,52,1,1,5,5,8.0,8727.797555,9234.055759,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,41481.0,1,21,4,3,4,4,6.0,24342.505253,24655.376656,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,41482.0,1,64,1,2,6,6,15.0,9811.075078,11602.178638,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29251,102949.0,1,33,3,3,5,5,6.0,29334.650205,30287.188731,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
29252,102952.0,2,70,5,3,2,2,4.0,16896.276203,18338.711104,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
29253,102953.0,1,42,1,3,1,1,12.0,61630.380013,63661.951573,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
29254,102954.0,2,41,4,5,7,7,10.0,17160.895269,17694.783346,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [87]:
for val in names:
    print(df[val].isna().sum())

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [88]:
ohe = OneHotEncoder(sparse=False)
j=0
for val in names:
    print(val)
    array =ohe.fit_transform(df[[val]])
    df_array = pd.DataFrame(array)
    for i in df_array.columns:
        name = val+'_'+str(j)
        print(name)
        print(df_array[i])
        df[name]= df_array[i]
        j+=1
    j=0

pusle
pusle_0
0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
29251    0.0
29252    0.0
29253    0.0
29254    0.0
29255    0.0
Name: 0, Length: 29256, dtype: float64
pusle_1
0        0.0
1        0.0
2        0.0
3        1.0
4        0.0
        ... 
29251    0.0
29252    0.0
29253    0.0
29254    0.0
29255    0.0
Name: 1, Length: 29256, dtype: float64
pusle_2
0        1.0
1        0.0
2        1.0
3        0.0
4        0.0
        ... 
29251    0.0
29252    1.0
29253    0.0
29254    0.0
29255    0.0
Name: 2, Length: 29256, dtype: float64
pusle_3
0        0.0
1        1.0
2        0.0
3        0.0
4        1.0
        ... 
29251    0.0
29252    0.0
29253    1.0
29254    1.0
29255    1.0
Name: 3, Length: 29256, dtype: float64
pusle_4
0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
29251    0.0
29252    0.0
29253    0.0
29254    0.0
29255    0.0
Name: 4, Length: 29256, dtype: float64
pusle_5
0        0.0
1        0.0
2        0

In [89]:
df

,SEQN,GENDER,AGE,ETHNICITY,EDUCATION,HOUSE_INDIV,FAMILY_SIZE,FAM_INCOME,INTERVIE_wgt,EXAM_wgt,...,LDL_0,LDL_1,LDL_2,LDL_3,LDL_4,LDL_5,FOLATE_0,FOLATE_1,FOLATE_2,FOLATE_3
0,41475.0,2,62,5,3,2,2,6.0,59356.356426,60045.772497,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,41477.0,1,71,3,3,2,2,5.0,9935.266183,10074.150074,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,41479.0,1,52,1,1,5,5,8.0,8727.797555,9234.055759,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,41481.0,1,21,4,3,4,4,6.0,24342.505253,24655.376656,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,41482.0,1,64,1,2,6,6,15.0,9811.075078,11602.178638,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29251,102949.0,1,33,3,3,5,5,6.0,29334.650205,30287.188731,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
29252,102952.0,2,70,5,3,2,2,4.0,16896.276203,18338.711104,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
29253,102953.0,1,42,1,3,1,1,12.0,61630.380013,63661.951573,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
29254,102954.0,2,41,4,5,7,7,10.0,17160.895269,17694.783346,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [90]:
df.to_csv("Data_After_Part2.csv")

In [91]:
df_numerics_only = df.select_dtypes(include=np.number)

In [92]:
impute_knn= KNNImputer()
df_numerics_only=pd.DataFrame(impute_knn.fit_transform(df_numerics_only))

In [93]:
df_numerics_only.isna().sum()

0      0
1      0
2      0
3      0
4      0
      ..
250    0
251    0
252    0
253    0
254    0
Length: 255, dtype: int64

In [94]:
df_numerics_only

,0,1,2,3,4,5,6,7,8,9,...,245,246,247,248,249,250,251,252,253,254
0,41475.0,2.0,62.0,5.0,3.0,2.0,2.0,6.0,59356.356426,60045.772497,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,41477.0,1.0,71.0,3.0,3.0,2.0,2.0,5.0,9935.266183,10074.150074,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,41479.0,1.0,52.0,1.0,1.0,5.0,5.0,8.0,8727.797555,9234.055759,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,41481.0,1.0,21.0,4.0,3.0,4.0,4.0,6.0,24342.505253,24655.376656,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,41482.0,1.0,64.0,1.0,2.0,6.0,6.0,15.0,9811.075078,11602.178638,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29251,102949.0,1.0,33.0,3.0,3.0,5.0,5.0,6.0,29334.650205,30287.188731,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
29252,102952.0,2.0,70.0,5.0,3.0,2.0,2.0,4.0,16896.276203,18338.711104,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
29253,102953.0,1.0,42.0,1.0,3.0,1.0,1.0,12.0,61630.380013,63661.951573,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
29254,102954.0,2.0,41.0,4.0,5.0,7.0,7.0,10.0,17160.895269,17694.783346,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [95]:
df

,SEQN,GENDER,AGE,ETHNICITY,EDUCATION,HOUSE_INDIV,FAMILY_SIZE,FAM_INCOME,INTERVIE_wgt,EXAM_wgt,...,LDL_0,LDL_1,LDL_2,LDL_3,LDL_4,LDL_5,FOLATE_0,FOLATE_1,FOLATE_2,FOLATE_3
0,41475.0,2,62,5,3,2,2,6.0,59356.356426,60045.772497,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,41477.0,1,71,3,3,2,2,5.0,9935.266183,10074.150074,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,41479.0,1,52,1,1,5,5,8.0,8727.797555,9234.055759,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,41481.0,1,21,4,3,4,4,6.0,24342.505253,24655.376656,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,41482.0,1,64,1,2,6,6,15.0,9811.075078,11602.178638,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29251,102949.0,1,33,3,3,5,5,6.0,29334.650205,30287.188731,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
29252,102952.0,2,70,5,3,2,2,4.0,16896.276203,18338.711104,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
29253,102953.0,1,42,1,3,1,1,12.0,61630.380013,63661.951573,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
29254,102954.0,2,41,4,5,7,7,10.0,17160.895269,17694.783346,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [96]:
colnames_numerics_only = df.select_dtypes(include=np.number).columns.tolist()
colnames_numerics_only

['SEQN',
 'GENDER',
 'AGE',
 'ETHNICITY',
 'EDUCATION',
 'HOUSE_INDIV',
 'FAMILY_SIZE',
 'FAM_INCOME',
 'INTERVIE_wgt',
 'EXAM_wgt',
 'HEIGHT',
 'WEIGHT',
 'BMI',
 'WAIST_CIRC',
 'age_first_smoke',
 'sw_cannabis',
 'sw_drugs',
 'HbA1c',
 'WBC',
 'lymphocyte_pct',
 'monocyte_pct',
 'neutrophil_pct',
 'eosinophil_pct',
 'basophil_pct',
 'lymphocyte_cnt',
 'monocyte_cnt',
 'neutrophil_cnt',
 'eosinophil_cnt',
 'basophil_cnt',
 'RBC',
 'hemoglobin',
 'hematocrit',
 'MCV',
 'MCH',
 'MCHC',
 'RDW',
 'platelets',
 'MPV',
 'sw_diab',
 'sw_prediab',
 'sw_diabrisk',
 'sw_insulin',
 'sw_oral_antidiab',
 'diab_retinopathy',
 'CAD',
 'CHF',
 'MI',
 'STROKE',
 'FAMILY_DIAB',
 'sw_asthma_ever',
 'sw_overweight',
 'sw_malignancy',
 'RENAL',
 'DIALYSIS',
 'FERRITIN',
 'BUN_CREATININE_0',
 'BUN_CREATININE_1',
 'BUN_CREATININE_2',
 'BUN_CREATININE_3',
 'ALBUMIN_CREATININE_0',
 'ALBUMIN_CREATININE_1',
 'ALBUMIN_CREATININE_2',
 'ALBUMIN_CREATININE_3',
 'URIC_CREATININE_0',
 'URIC_CREATININE_1',
 'URIC_CREA

In [97]:
df[val]

0         <61.2
1         <61.2
2         <61.2
3            no
4         <61.2
          ...  
29251    >385.2
29252        no
29253        no
29254    >385.2
29255    >385.2
Name: FOLATE, Length: 29256, dtype: object

In [98]:
i=0
for val in colnames_numerics_only:
    df[val]=df_numerics_only[i]
    i+=1

In [99]:
df.isna().sum()

SEQN         0
GENDER       0
AGE          0
ETHNICITY    0
EDUCATION    0
            ..
LDL_5        0
FOLATE_0     0
FOLATE_1     0
FOLATE_2     0
FOLATE_3     0
Length: 304, dtype: int64

In [100]:
df.to_csv("Data_After_knn.csv")

In [101]:
df['ALBUMIN_CREATININE_0'].value_counts()

1.0    14896
0.0    14360
Name: ALBUMIN_CREATININE_0, dtype: int64